In [2]:
import pandas as pd
import numpy as np
import torch
import PIL.Image as im
import torchvision
from tqdm import tqdm
import cv2

In [59]:
df=pd.read_csv("Multi-Label Retinal Diseases (MuReD) Dataset/train_data.csv")

In [60]:
valdf=pd.read_csv("Multi-Label Retinal Diseases (MuReD) Dataset/val_data.csv")

In [100]:
rf2_train= pd.read_csv("RFMiD2/Training_set/imputable_train_split.csv")
rf2_test=pd.read_csv("RFMiD2/Test_set/Cleand_test_data.csv")
rf2_val=pd.read_csv("RFMID2/Validation_set/Cleaned_validation_split.csv")

In [101]:
normal={"WNL":"NORMAL"}
rf2_test.rename(columns=normal, inplace=True)

In [102]:
rf2_train.rename(columns=normal, inplace=True)
rf2_val.rename(columns=normal, inplace=True)

In [103]:
rf2_test

,ID,NORMAL,AH,AION,ARMD,BRVO,CB,CF,CL,CME,...,RS,RT,SOFE,ST,TD,TSLN,TV,VS,HTN,IIH
0,167,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,189,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,191,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,201,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,216,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,896,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
167,897,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
168,898,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
169,899,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [104]:
a=set(df.columns).intersection(rf2_train.columns)

In [105]:
len(a)

18

In [106]:
other=set(rf2_test).difference(df)

In [107]:
set(df).difference(rf2_test)

{'ASR', 'HTR', 'OTHER'}

In [108]:
len(other)

34

In [109]:
def others(df,other):
    df["OTHER"]=df[df[other].any(1)].sum(axis=1)
    df["OTHER"]=df["OTHER"].apply(lambda x: 1 if x>0 else 0)
    df.drop(columns=other,inplace=True)
    df["HTR"]=0
    df["ASR"]=0
    return df

In [110]:
r2_train=others(rf2_train,other)

C:\Users\jagg_\AppData\Local\Temp\ipykernel_33436\1819099139.py:2: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df["OTHER"]=df[df[other].any(1)].sum(axis=1)


In [118]:
cols=df.columns.to_list()
r2_train=r2_train.reindex(columns=cols)
r2_test=r2_test.reindex(columns=cols)
r2_val=r2_val.reindex(columns=cols)

In [120]:
r2_train.to_csv("RFMiD2/Training_set/train_set.csv",index=False)
r2_test.to_csv("RFMiD2/Test_set/test_set.csv",index=False)
r2_val.to_csv("RFMID2/Validation_set/validation_set.csv",index=False)

In [114]:
r2_test=others(rf2_test,other)

r2_val=others(rf2_val,other)


C:\Users\jagg_\AppData\Local\Temp\ipykernel_33436\1819099139.py:2: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df["OTHER"]=df[df[other].any(1)].sum(axis=1)
C:\Users\jagg_\AppData\Local\Temp\ipykernel_33436\1819099139.py:2: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df["OTHER"]=df[df[other].any(1)].sum(axis=1)


In [119]:
r2_train.columns

Index(['ID', 'DR', 'NORMAL', 'MH', 'ODC', 'TSLN', 'ARMD', 'DN', 'MYA', 'BRVO',
       'ODP', 'CRVO', 'CNV', 'RS', 'ODE', 'LS', 'CSR', 'HTR', 'ASR', 'CRS',
       'OTHER'],
      dtype='object')

In [30]:
df.columns

Index(['ID', 'DR', 'NORMAL', 'MH', 'ODC', 'TSLN', 'ARMD', 'DN', 'MYA', 'BRVO',
       'ODP', 'CRVO', 'CNV', 'RS', 'ODE', 'LS', 'CSR', 'HTR', 'ASR', 'CRS',
       'OTHER'],
      dtype='object')

In [5]:
df

,ID,DR,NORMAL,MH,ODC,TSLN,ARMD,DN,MYA,BRVO,...,CRVO,CNV,RS,ODE,LS,CSR,HTR,ASR,CRS,OTHER
0,aria_c_25_1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,aria_c_7_2,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,aria_c_38_2,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,aria_c_2_8,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,aria_c_26_15,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1759,im0395,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1760,im0396,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
1761,im0398,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
1762,im0400,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1


In [10]:
x=df.drop(columns="ID").sum()
x=x[x<112].index.to_list()
x

['MYA',
 'BRVO',
 'ODP',
 'CRVO',
 'CNV',
 'RS',
 'ODE',
 'LS',
 'CSR',
 'HTR',
 'ASR',
 'CRS']

In [11]:
data=pd.read_csv("RFMiD2_0\Training_set\RFMiD_2_Training_labels.csv", encoding="latin")

In [16]:
a=set(data.columns).intersection(x)
data[a].sum()

C:\Users\jagg_\AppData\Local\Temp\ipykernel_18836\4388056.py:2: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  data[a].sum()


MYA     27
RS       5
LS      12
CNV      1
CSR     10
ODP     10
ODE     11
CRS     25
CRVO     7
BRVO    12
dtype: int64

In [17]:
df[a].sum()

C:\Users\jagg_\AppData\Local\Temp\ipykernel_18836\1912695457.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df[a].sum()


MYA     71
RS      47
LS      37
CNV     48
CSR     29
ODP     50
ODE     46
CRS     24
CRVO    44
BRVO    63
dtype: int64

In [6]:
img=im.open("Multi-Label Retinal Diseases (MuReD) Dataset/images/aria_c_25_1.tif")

In [6]:
import os
ls=os.listdir("Multi-Label Retinal Diseases (MuReD) Dataset/images")

In [8]:
model=torchvision.transforms.Resize((512,512),antialias=True)
path="Multi-Label Retinal Diseases (MuReD) Dataset/images/"
# model.to('cuda:0')
for i in tqdm(ls):
    img=cv2.imread(path+i)
#     img=model(torch.tensor(img.transpose((2,1,0))))
    cv2.imwrite(path+i.split(".")[0]+".png",img)

100%|██████████████████████████████████████████████████████████████████████████████| 2451/2451 [04:11<00:00,  9.74it/s]


In [7]:
ls=[i for i in ls if i.split('.')[-1]=='png']

In [8]:
len(ls)

2451

In [9]:
ls

['(0001)aria_d_26.png',
 '(0001)aria_d_27.png',
 '(0003)aria_d_29.png',
 '(0004)aria_d_28.png',
 '(0007)aria_d_28.png',
 '(0018)aria_d_26.png',
 '(0024)aria_d_30.png',
 '(0026)aria_d_27.png',
 '(0031)aria_c_37.png',
 '1.png',
 '10.png',
 '100.png',
 '1000.png',
 '1001.png',
 '1002.png',
 '1003.png',
 '1004.png',
 '1005.png',
 '1006.png',
 '1007.png',
 '1008.png',
 '1009.png',
 '101.png',
 '1010.png',
 '1011.png',
 '1012.png',
 '1013.png',
 '1014.png',
 '1015.png',
 '1016.png',
 '1017.png',
 '1018.png',
 '1019.png',
 '102.png',
 '1020.png',
 '1021.png',
 '1022.png',
 '1023.png',
 '1024.png',
 '1025.png',
 '1026.png',
 '1027.png',
 '1028.png',
 '1029.png',
 '103.png',
 '1030.png',
 '1031.png',
 '1032.png',
 '1033.png',
 '1034.png',
 '1035.png',
 '1036.png',
 '1037.png',
 '1038.png',
 '1039.png',
 '104.png',
 '1040.png',
 '1041.png',
 '1042.png',
 '1043.png',
 '1044.png',
 '1045.png',
 '1046.png',
 '1047.png',
 '1048.png',
 '1049.png',
 '105.png',
 '1050.png',
 '1051.png',
 '1052.png',
 '

In [10]:
df.shape[0]+valdf.shape[0]

2208

In [11]:
model=torchvision.transforms.Resize((512,512),antialias=True)
path="Multi-Label Retinal Diseases (MuReD) Dataset/images/"
out_path="Multi-Label Retinal Diseases (MuReD) Dataset/Data/"
# model.to('cuda:0')
for i in tqdm(ls):
    img=im.open(path+i)
    img=model(img)
    img.save(out_path+i)

100%|██████████████████████████████████████████████████████████████████████████████| 2451/2451 [06:19<00:00,  6.47it/s]


In [16]:
ls[348]

'1303.png'